# CHOCOLATE BAR RATINGS

In [50]:
# Importing libraries

import pandas as pd
import numpy as np
import numpy.linalg as nla
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re
import math
import six

In [52]:
# Changing the format of the values visualization

pd.options.display.float_format = '{:.2f}'.format

# Reseting the maximum number of columns to display to default (all columns)

pd.set_option('display.max_columns', None)

# Importing Data

In [55]:
path = r'C:\Users\IVON6\Chocolate Bar Ratings Analysis'
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'flavors_of_cacao.csv'))

In [57]:
# Print the first 5 rows
df.head()

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


In [59]:
df.dtypes

Company \n(Maker-if known)            object
Specific Bean Origin\nor Bar Name     object
REF                                    int64
Review\nDate                           int64
Cocoa\nPercent                        object
Company\nLocation                     object
Rating                               float64
Bean\nType                            object
Broad Bean\nOrigin                    object
dtype: object

# Cleaning Data

In [62]:
# Null analysis
df.isnull().sum()

Company \n(Maker-if known)           0
Specific Bean Origin\nor Bar Name    0
REF                                  0
Review\nDate                         0
Cocoa\nPercent                       0
Company\nLocation                    0
Rating                               0
Bean\nType                           1
Broad Bean\nOrigin                   1
dtype: int64

In [64]:
# Duplicates analysis
df.duplicated().sum()

0

In [66]:
 # Rename the columns.

 df.columns = [
        'company', 'bar_name', 'reference_number',
        'review_date', 'cocoa_percent', 'company_location',
        'rating', 'bean_type', 'bean_origin'
    ]


In [68]:
df.dtypes

company              object
bar_name             object
reference_number      int64
review_date           int64
cocoa_percent        object
company_location     object
rating              float64
bean_type            object
bean_origin          object
dtype: object

In [70]:
  # Replace empty/null values with "Blend"
df['bean_type'] = df['bean_type'].fillna('Blend')

In [72]:
# Null analysis
df.isnull().sum()

company             0
bar_name            0
reference_number    0
review_date         0
cocoa_percent       0
company_location    0
rating              0
bean_type           0
bean_origin         1
dtype: int64

In [74]:
# Convert the column to numeric values (floats)
df['cocoa_percent'] = pd.to_numeric(df['cocoa_percent'].str.rstrip('%'), errors='coerce')

# Remove NaN values and convert to integers
df['cocoa_percent'] = df['cocoa_percent'].fillna(0).astype(int)

# Remove leading and trailing spaces from 'bean_type' column
df['bean_type'] = df['bean_type'].str.strip()

# Replace empty strings with 'Missing'
df['bean_type'] = df['bean_type'].replace('', 'Missing')

# Get the top values of 'bean_type'
top_bean_types = df['bean_type'].value_counts().head(10)

# Create a DataFrame of top 'Bean_Type' values and their counts
top_bean_types_df = pd.DataFrame({'bean_type': top_bean_types.index, 'Count': top_bean_types.values})

# Create a Styler object to highlight 'Missing' values in the table
def highlight_missing(s):
    is_missing = s == 'Missing'
    return ['background-color: yellow' if v else '' for v in is_missing]

styled_table = top_bean_types_df.style.apply(highlight_missing)

# Display the styled table
styled_table

,bean_type,Count
0,Missing,887
1,Trinitario,419
2,Criollo,153
3,Forastero,87
4,Forastero (Nacional),52
5,Blend,42
6,"Criollo, Trinitario",39
7,Forastero (Arriba),37
8,Criollo (Porcelana),10
9,"Trinitario, Criollo",9


In [76]:
df.describe()

,reference_number,review_date,cocoa_percent,rating
count,1795.00,1795.00,1795.00,1795.00
mean,1035.90,2012.33,71.70,3.19
std,552.89,2.93,6.32,0.48
min,5.00,2006.00,42.00,1.00
25%,576.00,2010.00,70.00,2.88
50%,1069.00,2013.00,70.00,3.25
75%,1502.00,2015.00,75.00,3.50
max,1952.00,2017.00,100.00,5.00


In [78]:
df.shape

(1795, 9)

In [80]:
# Correct spelling mistakes, and replace city with country name

df['company_location'] = df['company_location']\
.str.replace('Amsterdam', 'Holland')\
.str.replace('U.K.', 'England')\
.str.replace('Niacragua', 'Nicaragua')\
.str.replace('Domincan Republic', 'Dominican Republic')

In [82]:
  # Adding this so that Holland and Netherlands map to the same country.
df['company_location'] = df['company_location']\
.str.replace('Holland', 'Netherlands')

In [84]:
       def cleanup_spelling_abbrev(text):
        replacements = [
            ['-', ', '], ['/ ', ', '], ['/', ', '], ['\\(', ', '], [' and', ', '], [' &', ', '], ['\\)', ''],
            ['Dom Rep|DR|Domin Rep|Dominican Rep,|Domincan Republic', 'Dominican Republic'],
            ['Mad,|Mad$', 'Madagascar, '],
            ['PNG', 'Papua New Guinea, '],
            ['Guat,|Guat$', 'Guatemala, '],
            ['Ven,|Ven$|Venez,|Venez$', 'Venezuela, '],
            ['Ecu,|Ecu$|Ecuad,|Ecuad$', 'Ecuador, '],
            ['Nic,|Nic$', 'Nicaragua, '],
            ['Cost Rica', 'Costa Rica'],
            ['Mex,|Mex$', 'Mexico, '],
            ['Jam,|Jam$', 'Jamaica, '],
            ['Haw,|Haw$', 'Hawaii, '],
            ['Gre,|Gre$', 'Grenada, '],
            ['Tri,|Tri$', 'Trinidad, '],
            ['C Am', 'Central America'],
            ['S America', 'South America'],
            [', $', ''], [',  ', ', '], [', ,', ', '], ['\xa0', ' '],[',\\s+', ','],
            [' Bali', ',Bali']
        ]
        for i, j in replacements:
            text = re.sub(i, j, text)
        return text

df['bar_name'] = df['bar_name'].str.replace('.', '').apply(cleanup_spelling_abbrev)

In [86]:
# Cast specific_origin to string

df['bar_name'] = df['bar_name'].astype(str)

In [88]:
# Replace null-valued fields with the same value as for specific_origin
df['bean_origin'] = df['bean_origin'].fillna(df['bar_name'])

In [90]:
 # Clean up spelling mistakes and deal with abbreviations
df['bean_origin'] = df['bean_origin'].str.replace('.', '').apply(cleanup_spelling_abbrev)

In [92]:
 # Change 'Trinitario, Criollo' to "Criollo, Trinitario"
# Check with df['bean_type'].unique()
df.loc[df['bean_type'].isin(['Trinitario, Criollo']),'bean_type'] = "Criollo, Trinitario"

In [94]:
# Fix chocolate maker names

df.loc[df['company']=='Shattel','company'] = 'Shattell'
df['company'] = df['company'].str.replace(u'Na\xef\xbf\xbdve','Naive')

In [96]:
df.head()

,company,bar_name,reference_number,review_date,cocoa_percent,company_location,rating,bean_type,bean_origin
0,A. Morin,Agua Grande,1876,2016,63,France,3.75,Missing,Sao Tome
1,A. Morin,Kpime,1676,2015,70,France,2.75,Missing,Togo
2,A. Morin,Atsane,1676,2015,70,France,3.00,Missing,Togo
3,A. Morin,Akata,1680,2015,70,France,3.50,Missing,Togo
4,A. Morin,Quilla,1704,2015,70,France,3.50,Missing,Peru


In [98]:
df.shape

(1795, 9)

In [100]:
print(f'The column names are: {list(df.columns)}.\n')



The column names are: ['company', 'bar_name', 'reference_number', 'review_date', 'cocoa_percent', 'company_location', 'rating', 'bean_type', 'bean_origin'].



In [102]:
for col in list(df.columns):
  print(f'The unique value of column "{col}" are: {df[col].nunique()}.')

The unique value of column "company" are: 415.
The unique value of column "bar_name" are: 1038.
The unique value of column "reference_number" are: 440.
The unique value of column "review_date" are: 12.
The unique value of column "cocoa_percent" are: 42.
The unique value of column "company_location" are: 58.
The unique value of column "rating" are: 13.
The unique value of column "bean_type" are: 40.
The unique value of column "bean_origin" are: 97.


In [104]:
# Dividing features into Numerical and Categorical
col = list(df.columns)
categorical_features = []
numerical_features = []
for i in col:
    # define a feature is categorical when it has less than 6 unique values ( get rid of missclasifying encoded categrocial feature )
    if len(df[i].unique()) > 6:
        numerical_features.append(i) 
    else:
        categorical_features.append(i) 

print('Categorical Features :',*categorical_features)
print('Numerical Features :',*numerical_features)

Categorical Features :
Numerical Features : company bar_name reference_number review_date cocoa_percent company_location rating bean_type bean_origin


In [106]:
df.describe()

,reference_number,review_date,cocoa_percent,rating
count,1795.00,1795.00,1795.00,1795.00
mean,1035.90,2012.33,71.70,3.19
std,552.89,2.93,6.32,0.48
min,5.00,2006.00,42.00,1.00
25%,576.00,2010.00,70.00,2.88
50%,1069.00,2013.00,70.00,3.25
75%,1502.00,2015.00,75.00,3.50
max,1952.00,2017.00,100.00,5.00


In [108]:
# Define the ranges for cocoa percentages
bins = [0, 50, 60, 70, 80, 90, 100]  # You can adjust the ranges as needed
# Define labels for the ranges
labels = ['<50%','50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
# Convert Cocoa Percent values into ranges
df['Cocoa_Percent_Range'] = pd.cut(df['cocoa_percent'], bins=bins, labels=labels, right=False)

In [110]:
df.to_csv(os.path.join(path, '02 Data','Prepared Data', 'flavor_of_cacao_clean.csv'))